In [1]:
import numpy as np
import pandas as pd
import requests
import math

In [2]:
from secrets import IEX_CLOUD_API_TOKEN

In [3]:
stocks = pd.read_csv("sp-500-stocks.csv")

In [4]:
columns = ["Tickers", "Stock Price", "Market Capitalization", "Number of Shares to Buy"]
final_dataframe = pd.DataFrame(columns=columns)

In [7]:
final_dataframe = pd.DataFrame(columns=columns)
for stock in stocks["Ticker"][:20]:
    api_url = f"https://sandbox.iexapis.com/stable/stock/{stock}/quote?token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
        pd.Series(
            [stock, data["latestPrice"], data["marketCap"], "N/A"], 
            index=columns
        ),
        ignore_index=True
    )

In [6]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [9]:
symbol_groups = list(chunks(stocks["Ticker"], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(",".join(symbol_groups[i]))
    # print(symbol_strings[i])
final_dataframe = pd.DataFrame(columns=columns)
for symbol_string in symbol_strings:
    batch_api_call_url = \
        f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_api_call_url)
    print(data.status_code)
    for symbol in symbol_string.split(","):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol, 
                    data[symbol]["quote"]["latestPrice"], 
                    data[symbol]["quote"]["marketCap"],
                    "N/A"
                ], 
                index=columns
            ), 
            ignore_index=True
        )
final_dataframe

AttributeError: 'dict' object has no attribute 'status_code'

In [36]:
portfolio_size = input("Enter the value of your portfolio:")
try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number!\nPlease try again ...")
    portfolio_size = input("Enter the value of your portfolio:")
    val = float(portfolio_size)

In [37]:
position_size = val / len(final_dataframe.index)
number_of_apple_shares = position_size / 500
print(math.floor(number_of_apple_shares))

39
